In [1]:
import os
import tqdm

import rasterio
import rasterio.plot
import rasterio.merge
import rasterio.mask
import shapely

# from reproject_raster import reproject_raster
from microsoft_planetarycomputer_download.utils import read_geojson
from utils.rasterio_utils import mask_raster_by_shapely, reproject_raster_v2

In [2]:
DATA_PATH = '/data/microsoft_planetary_computer'
PLACE_NAME = 'castaic_lake'
FORCE_REPROJECT = True
LABEL_RASTER_PATH = '/data/microsoft_planetary_computer/label_mosaic_v2.tiff'

In [3]:
GEOJSON_FILEPATH = 'geojsons/{}.geojson'.format(PLACE_NAME)

DEM_FILEPATH = os.path.join(DATA_PATH, 'dem', PLACE_NAME, 'mosaic', 'mosaic.tiff')
DEM_1M_FILEPATH = os.path.join(DATA_PATH, 'dem', PLACE_NAME, 'mosaic_1m', 'mosaic.tiff')

DSM_FILEPATH = os.path.join(DATA_PATH, 'dsm', PLACE_NAME, 'mosaic', 'mosaic.tiff')
NAIP_FILEPATH = os.path.join(DATA_PATH, 'naip', PLACE_NAME, 'mosaic', 'mosaic.tiff')

# Create output folders
OUTPUT_FOLDER = os.path.join(DATA_PATH, 'outputs', 'preprocessed')
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [4]:
# Read raster data (dynamic world labels)
dw_labels = rasterio.open(LABEL_RASTER_PATH)

In [5]:
# outputs/preprocessed/{epsg-xxxxx}/{place_name}/naip/{GSD}/mosaic.tiff
# outputs/preprocessed/{epsg-xxxxx}/{place_name}/dem/{GSD}/mosaic.tiff
# outputs/preprocessed/{epsg-xxxxx}/{place_name}/dsm/{GSD}/mosaic.tiff
# outputs/preprocessed/{epsg-xxxxx}/{place_name}/dynamicworld/{GSD}/mosaic.tiff

# Reproject naip/dsm/dem according to DynamicWorld label
for spatial_res in [0.6, 2.0, 10.0]: # meter / pixel
    target_polygon = shapely.geometry.box(*dw_labels.bounds)
    target_crs = dw_labels.crs
    epsg_code = target_crs.to_epsg()

    res_folder = '{:.1f}'.format(spatial_res) if spatial_res is not None else 'original_resolution'
    naip_reproj_path = os.path.join(OUTPUT_FOLDER, 'epsg-{}'.format(epsg_code), PLACE_NAME, 'naip', res_folder, 'mosaic.tiff')
    dem_reproj_path = os.path.join(OUTPUT_FOLDER, 'epsg-{}'.format(epsg_code), PLACE_NAME, 'dem', res_folder, 'mosaic.tiff')
    dsm_reproj_path = os.path.join(OUTPUT_FOLDER, 'epsg-{}'.format(epsg_code), PLACE_NAME, 'dsm', res_folder, 'mosaic.tiff')
    dw_reproj_path = os.path.join(OUTPUT_FOLDER, 'epsg-{}'.format(epsg_code), PLACE_NAME, 'dynamicworld', res_folder, 'mosaic.tiff')

    path_dict = {
        'naip': {'original' : NAIP_FILEPATH, 'reprojected': naip_reproj_path, 'interpolation' : rasterio.enums.Resampling.cubic},
        'dem': {'original' : DEM_FILEPATH, 'reprojected': dem_reproj_path, 'interpolation' : rasterio.enums.Resampling.bilinear},
        'dem_1m': {'original' : DEM_1M_FILEPATH, 'reprojected': dem_reproj_path, 'interpolation' : rasterio.enums.Resampling.bilinear},
        'dsm': {'original' : DSM_FILEPATH, 'reprojected': dsm_reproj_path, 'interpolation' : rasterio.enums.Resampling.cubic},
        'dw': {'original' : LABEL_RASTER_PATH, 'reprojected': dw_reproj_path, 'interpolation' : rasterio.enums.Resampling.nearest},
    }

    os.makedirs(os.path.dirname(naip_reproj_path), exist_ok=True)
    os.makedirs(os.path.dirname(dem_reproj_path), exist_ok=True)
    os.makedirs(os.path.dirname(dsm_reproj_path), exist_ok=True)
    os.makedirs(os.path.dirname(dw_reproj_path), exist_ok=True)

    for name, obj in path_dict.items():
        reprojected = obj['reprojected']
        original = obj['original']
        if not os.path.exists(reprojected) or FORCE_REPROJECT:
            reproject_raster_v2(original, reprojected, dst_crs=target_crs, spatial_res=spatial_res, interpolation=obj['interpolation'])
            mask_raster_by_shapely(reprojected, reprojected, target_polygon)

Reprojecting and resampling raster...
Input: /data/microsoft_planetary_computer/naip/castaic_lake/mosaic/mosaic.tiff (35940, 39470) EPSG:26911 metre (0.6, 0.6)
Output: /data/microsoft_planetary_computer/outputs/preprocessed/epsg-32611/castaic_lake/naip/0.6/mosaic.tiff (35940, 39470) EPSG:32611 metre (0.6, 0.6)
Masking (cropping) raster...
Applying crop mask to raster...
Input: /data/microsoft_planetary_computer/outputs/preprocessed/epsg-32611/castaic_lake/naip/0.6/mosaic.tiff (35940, 39470) EPSG:32611 metre (0.6, 0.6)
Output: /data/microsoft_planetary_computer/outputs/preprocessed/epsg-32611/castaic_lake/naip/0.6/mosaic.tiff (27021, 29349) EPSG:32611 metre (0.6, 0.6)
Reprojecting and resampling raster...
